In [34]:
import pandas as pd
import numpy as np

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# kmeans
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import requests

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print('Libraries imported.')

Libraries imported.


In [35]:
## Read Data from wikipedia

In [36]:

link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

tables = pd.read_html(link,header=0)

## Rename the column Neighburhood to Neighborhood in the dataframe

In [37]:
df = tables[0]
df.rename(columns={'Neighbourhood': 'Neighborhood'}, inplace=1)


In [38]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Assign the value of Borough to Neighborhood for any role where the Neighborhood value is 'Not assigned'. Only one row in the dataframe has such

In [39]:
df.loc[df['Neighborhood']=="Not assigned",'Neighborhood']=df.loc[df['Neighborhood']=="Not assigned",'Borough']

In [40]:
df.shape

(288, 3)

## Replace value 'Not assigned' to 'NaN' so we can drop rows without a value

In [41]:
df.replace('Not assigned', np.NaN, inplace=True)

In [42]:
df = df.dropna()
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## Remove duplicate Postcode by combining duplicates row as one entry

In [44]:
df = pd.DataFrame(df.groupby(['Postcode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index())
                    

In [45]:
#!conda install -c conda-forge geocoder --yes
import geocoder

### Using geocoder with arcgis to get the location coordinates. For some reason geocoder.google won't connect

In [46]:
def get_coords(row):
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(row))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [47]:
coords = df['Postcode'].apply(get_coords).tolist()
coords

[[43.811525000000074, -79.19551721399995],
 [43.78573000000006, -79.15874999999994],
 [43.76569000000006, -79.17525603599995],
 [43.76835912100006, -79.21758999999997],
 [43.76968799900004, -79.23943999999995],
 [43.74312500000008, -79.23174973599998],
 [43.726244585000074, -79.26366999999993],
 [43.71313321100007, -79.28505499999994],
 [43.72357500000004, -79.23497617799995],
 [43.69666500000005, -79.26016331599999],
 [43.759975000000054, -79.26897402899993],
 [43.750710464000065, -79.30055999999996],
 [43.79394000000008, -79.26798280099996],
 [43.78472500000004, -79.29904659999994],
 [43.817810000000065, -79.28024362199994],
 [43.80088094900003, -79.32073999999994],
 [43.83421500000003, -79.21670085099998],
 [43.80284500000005, -79.35623615099996],
 [43.780880000000025, -79.34779577599994],
 [43.781015000000025, -79.38054242199996],
 [43.75719200000003, -79.37986499999994],
 [43.79147500000005, -79.41360487299994],
 [43.76816500000007, -79.40741984599998],
 [43.74785500000007, -79.40

### Add the columns Latitude and Longitude to the dataframe

In [48]:
df[['Latitude','Longitude']] = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [49]:
df

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.696665,-79.260163


In [50]:
df.shape

(103, 5)